In [ ]:
import os
import sys
import gc
import pandas as pd
import pandas_ta as ta
from math import pi
from bokeh.plotting import figure, show
import joblib
import m2cgen as m2c

sys.path.append(os.getcwd() + os.sep + '..' + os.sep + '..' + os.sep + '..')

from pyautotrader.utils.model_export import export_model_python

In [ ]:
data_folder = os.path.join('..','00.data','output')

files_found = [x for x in os.listdir(data_folder) if x.endswith('.pickle')]
files_found_tokens=[x.split('.') for x in files_found]

models_found = {}

for current_model in files_found_tokens:
    model_name = '.'.join(current_model[:-2])
    filename = os.path.join('..','00.data','output','.'.join(current_model))
    if not model_name in models_found:
        models_found[model_name]= {}          
    models_found[model_name][current_model[-2]]=filename

print(models_found)

In [ ]:
first_model = list(models_found.keys())[0]
current_total_dataset = joblib.load(models_found[first_model]['total'])
current_parameters = joblib.load(models_found[first_model]['parameters'])
current_raw_dataset = joblib.load(models_found[first_model]['raw'])

best_short_booster = joblib.load(models_found[first_model]['xgboostshortmodel'])
best_long_booster = joblib.load(models_found[first_model]['xgboostlongmodel'])

In [ ]:
df_current_total_dataset = pd.DataFrame(current_total_dataset)

In [ ]:
df_current_total_dataset = df_current_total_dataset[['current_date', 'current_time'] + current_parameters['CURRENT_X_COLUMNS']]

In [ ]:
current_total_dataset = None
gc.collect()

In [ ]:
def predict_short(row):
    a = row[current_parameters['CURRENT_X_COLUMNS']].to_numpy().reshape(1,-1)
    return best_short_booster.inplace_predict(a)[0]

def predict_long(row):
    a = row[current_parameters['CURRENT_X_COLUMNS']].to_numpy().reshape(1,-1)
    return best_long_booster.inplace_predict(a)[0]

df_current_total_dataset['short_predict'] = df_current_total_dataset.apply( lambda row: predict_short(row), axis=1)
df_current_total_dataset['long_predict'] = df_current_total_dataset.apply( lambda row: predict_long(row), axis=1)

In [ ]:
df_current_total_dataset['short_predict'].hist(bins=500)

In [ ]:
df_current_total_dataset['long_predict'].hist(bins=500)

In [ ]:
results_from_df = df_current_total_dataset.to_dict('records')
results = {}
short_results = []
long_results = []

for current_result in results_from_df:
    results[(current_result['current_date'] * 10000) + current_result['current_time']] = \
    { 
        'short_predict' : current_result['short_predict'],
        'long_predict' : current_result['long_predict']
    }    
    short_results.append(current_result['short_predict'])
    long_results.append(current_result['long_predict'])

#print(results)    

In [ ]:
minimum_short_predict = 0
minimum_long_predict = 0

maximum_short_predict = int(round(max(short_results),0))
maximum_long_predict = int(round(max(long_results),0))

current_short_predict = 0
current_long_predict = 0

current_trade = None
current_trade_entries = []

for current_candle in current_raw_dataset:
    if current_trade is None:
        if current_candle['Date'] > current_parameters['MINIMUM_DATE_TRADE'] and \
           current_candle['Time'] >= current_parameters['MINIMUM_TIME'] and \
           current_candle['Time'] <= current_parameters['MAXIMUM_TIME']:
            current_date_time = (current_candle['Date'] * 10000) + current_candle['Time']
            if results[current_date_time]['short_predict'] > 0:
                current_trade = {**current_candle, 'trade_type':'short'}
                current_trade_entries.append(current_trade)
            if results[current_date_time]['long_predict'] > 0:
                current_trade = {**current_candle, 'trade_type':'long'}
                current_trade_entries.append(current_trade)

In [ ]:
code = export_model_python()
print(code)